In [1]:
from ccdc.io import CrystalReader

crystal_reader = CrystalReader(r'C:\Users\Clamfighter\Machine_Learning_Project\my_env\Masters\T2_9_experimental_crystals\T2_9_experimental_crystals\DEBXIT01_1478358_T2gamma_240K.cif', format='cif')
cryst0 = crystal_reader[0]
crystal_reader = CrystalReader(r'C:\Users\Clamfighter\Machine_Learning_Project\my_env\Masters\T2_9_experimental_crystals\T2_9_experimental_crystals\T2epsilon.cif', format='cif')
cryst1 = crystal_reader[0]

In [2]:
from ccdc.crystal import PackingSimilarity
PS = PackingSimilarity()
PS.settings.packing_shell_size = 10
comp = PS.compare(cryst0,cryst1)
print(comp.rmsd)
print(comp.nmatched_molecules)
print(comp.dis)

0.1833274334455433
3


AttributeError: 'Comparison' object has no attribute 'dis'

In [5]:
import os
import time
'''
Initial run 9x9 with duplicate comparisons: 516.1s

'''
directory = r'C:\Users\Clamfighter\Machine_Learning_Project\my_env\Masters\T2_9_experimental_crystals\T2_9_experimental_crystals'
PS = PackingSimilarity()
i=0
starttime = time.time()
print(starttime)
for outter_cif in os.listdir(directory):
    #print(i)
    #print(outter_cif)

    crystal_reader = CrystalReader(directory+'\\'+outter_cif)
    outter_crystal = crystal_reader[0]

    for inner_cif in os.listdir(directory):
        if inner_cif != outter_cif:
            #print(inner_cif)

            crystal_reader = CrystalReader(directory+'\\'+inner_cif)
            inner_crystal = crystal_reader[0]
            comp = PS.compare(outter_crystal,inner_crystal)
            #print(comp.rmsd)
        #else:
            #print('match')
    i+=1
endtime = time.time()
print(endtime)
print(endtime - starttime)

1630358690.718377


KeyboardInterrupt: 

In [1]:
import os
import numpy as np
import pandas as pd
from ccdc.io import CrystalReader
from ccdc.crystal import PackingSimilarity
import time
'''
Initial run 9x9 with duplicate comparisons: 516.1s

Run without duplicate comparison 270.4s
'''
directory = r'C:\Users\Clamfighter\Machine_Learning_Project\my_env\Masters\T2_9_experimental_crystals\T2_9_experimental_crystals'
PS = PackingSimilarity()
#PS.settings.packing_shell_size = 15
#PS.settings.distance_tolerance = 0.02
PS.settings.angle_tolerance = 16
print(PS.settings.distance_tolerance)
print(PS.settings.angle_tolerance)
i=0
cifs = []


for filename in os.listdir(directory):
    cifs.append(filename)
#rmsd_matrix = np.empty((len(cifs),len(cifs)))
rmsd_matrix = np.empty((9,9))
rmsd_matrix[:] = np.nan
matchedMolecules_matrix = np.empty((len(cifs),len(cifs)))
matchedMolecules_matrix[:] = np.nan
cols = cifs
starttime = time.time()
print(starttime)
for i in range(len(cifs)):
    outter_cif = cifs[i]

    crystal_reader = CrystalReader(directory+'\\'+outter_cif)
    outter_crystal = crystal_reader[0]

    for j in range(len(cifs)):
        inner_cif = cifs[j]

        if inner_cif != outter_cif and inner_cif != "":
            crystal_reader = CrystalReader(directory+'\\'+inner_cif)
            inner_crystal = crystal_reader[0]
            comp = PS.compare(outter_crystal,inner_crystal)
            rmsd = comp.rmsd
            matched_molecules = comp.nmatched_molecules
            rmsd_matrix[i][j] = rmsd
            matchedMolecules_matrix[i][j] = matched_molecules

        elif inner_cif == outter_cif:
            rmsd_matrix[i][j] = 0
            matchedMolecules_matrix[i][j] = 15
    cifs[i] = ""
    i+=1
rmsd_df = pd.DataFrame(rmsd_matrix, columns=cols, index=cols)
matchedMolecule_df = pd.DataFrame(matchedMolecules_matrix, columns=cols, index=cols)
endtime = time.time()

print(rmsd_df)
print(matchedMolecule_df)
print(endtime)
print(endtime - starttime)

0.2
16.0
1630366345.271531
                                                                             \
  0.0  0.008312  0.003178  0.034960  0.451601  0.439513  0.186498  0.426636   
  NaN  0.000000  0.007424  0.034131  0.452272  0.440206  0.186618  0.426416   
  NaN       NaN  0.000000  0.033081  0.453086  0.440990  0.186980  0.427092   
  NaN       NaN       NaN  0.000000  0.462409  0.450179  0.186841  0.427149   
  NaN       NaN       NaN       NaN  0.000000  0.054768  0.246310  0.664421   
  NaN       NaN       NaN       NaN       NaN  0.000000  0.239013  0.654570   
  NaN       NaN       NaN       NaN       NaN       NaN  0.000000  0.273618   
  NaN       NaN       NaN       NaN       NaN       NaN       NaN  0.000000   
  NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

            
  0.183327  
  0.183868  
  0.183369  
  0.183587  
  0.434904  
  0.426874  
  0.231050  
  0.575751  
  0.000000  
                                                    

In [31]:
print(PS.settings.distance_tolerance)

0.2


In [2]:
cols = []
for filename in os.listdir(directory):
    cols.append(filename)
rmsd_df.index = cols
rmsd_df.columns = cols
matchedMolecule_df.index = cols
matchedMolecule_df.columns = cols
#print(df)
rmsd_df.to_csv(r"C:\Users\Clamfighter\Machine_Learning_Project\my_env\Masters\T2_9_experimental_crystals\fullComparisonOutputRMSD.csv")
matchedMolecule_df.to_csv(r"C:\Users\Clamfighter\Machine_Learning_Project\my_env\Masters\T2_9_experimental_crystals\fullComparisonOutputMatchedMolecules.csv")

In [3]:
import os
import numpy as np
import pandas as pd
from ccdc.io import CrystalReader
from ccdc.crystal import PackingSimilarity
import time
def RMSD(directory,PSS,DT,AT):
    #CCDC API Packing Similarity tool
    PS = PackingSimilarity()
    #defining parameters
    PS.settings.packing_shell_size = PSS
    PS.settings.distance_tolerance = DT
    PS.settings.angle_tolerance = AT
    i=0
    #get a list of all cifs in the directory
    cifs = []
    for filename in os.listdir(directory):
        cifs.append(filename)
    #create ouput matrices for RMSD and MM. their size depends on the number of CIFS
    rmsd_matrix = np.empty((len(cifs),len(cifs)))
    rmsd_matrix[:] = np.nan
    matchedMolecules_matrix = np.empty((len(cifs),len(cifs)))
    matchedMolecules_matrix[:] = np.nan
    #CIFs list will be altered. save CIFs to column list to be used in ouput later
    cols = cifs
    #begin iterative process & start timer
    starttime = time.time()
    print(starttime)
    for i in range(len(cifs)):
        '''
        Each CIF file is compared to all other CIF files
        This is done by creating two crystal reader objects, and
        comparing them with the packing shell comparison method.
        The RMSD and MM from the comparison is saved in the ouput
        matrices
        the CIF is then deleted from the CIFs list so that no comparison
        is repeated.
        '''
        outter_cif = cifs[i]
        crystal_reader = CrystalReader(directory+'\\'+outter_cif)
        outter_crystal = crystal_reader[0]
        for j in range(len(cifs)):
            inner_cif = cifs[j]

            if inner_cif != outter_cif and inner_cif != "":
                crystal_reader = CrystalReader(directory+'\\'+inner_cif)
                inner_crystal = crystal_reader[0]
                comp = PS.compare(outter_crystal,inner_crystal)#compares the two crystal objects
                rmsd = comp.rmsd
                matched_molecules = comp.nmatched_molecules
                rmsd_matrix[i][j] = rmsd
                matchedMolecules_matrix[i][j] = matched_molecules

            elif inner_cif == outter_cif: #if the two CIFs are the same, skip the comparison step
                rmsd_matrix[i][j] = 0
                matchedMolecules_matrix[i][j] = PSS
        cifs[i] = ""
        i+=1
    rmsd_df = pd.DataFrame(rmsd_matrix, columns=cols, index=cols)
    matchedMolecule_df = pd.DataFrame(matchedMolecules_matrix, columns=cols, index=cols)
    endtime = time.time()
    #Print outputs
    print(rmsd_df)
    print(matchedMolecule_df)
    print(endtime)
    print(endtime - starttime)
    rmsd_df.index = cols
    rmsd_df.columns = cols
    matchedMolecule_df.index = cols
    matchedMolecule_df.columns = cols
    #print(df)
    rmsd_df.to_csv(directory+'\output_RMSD.csv')
    matchedMolecule_df.to_csv(directory+'\output_MM.csv')
directory = r'C:\Users\Clamfighter\Machine_Learning_Project\my_env\Masters\T2_9_experimental_crystals\T2_9_experimental_crystals'
RMSD(directory,15,20,0.2)

<>:78: DeprecationWarning: invalid escape sequence \o
<>:79: DeprecationWarning: invalid escape sequence \o
<>:78: DeprecationWarning: invalid escape sequence \o
<>:79: DeprecationWarning: invalid escape sequence \o
<>:78: DeprecationWarning: invalid escape sequence \o
<>:79: DeprecationWarning: invalid escape sequence \o
<ipython-input-3-3afe6ae55cf9>:78: DeprecationWarning: invalid escape sequence \o
  rmsd_df.to_csv(directory+'\output_RMSD.csv')
<ipython-input-3-3afe6ae55cf9>:79: DeprecationWarning: invalid escape sequence \o
  matchedMolecule_df.to_csv(directory+'\output_MM.csv')
